<a href="https://colab.research.google.com/github/soham7707/DeepSectorAI/blob/main/GRU_GARCH_BETA__Bullish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 5.9 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib


In [3]:
# Define your sectors and representative stocks
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime

start_date = "2020-04-01"    #Bullish period
end_date = "2021-10-01"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-4217531400.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-4217531400.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-4217531400.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-4217531400.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-4217531400.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


handling missing data


In [5]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()  # Forward and backward fill
    sector_data[sector] = df


Calculating Log Returns

In [6]:
import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [7]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")


In [8]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2020-04-01,357.158722,787.779785,300.940430,1176.734009
2020-04-03,324.096832,772.777161,277.244293,1136.103638
2020-04-07,387.233154,850.876221,315.399841,1193.065674
2020-04-08,389.722748,844.039673,308.484467,1182.708984
2020-04-09,418.402985,878.365234,331.455200,1267.604370
...,...,...,...,...
2021-09-24,794.731689,1527.305908,701.275635,2020.665161
2021-09-27,793.636292,1549.764160,708.114258,2025.895996
2021-09-28,785.918518,1540.180054,695.649475,2060.618164


In [9]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2020-04-03,-0.097138,-0.019228,-0.082013,-0.035138
2020-04-07,0.177985,0.096276,0.128942,0.048922
2020-04-08,0.006409,-0.008067,-0.022170,-0.008719
2020-04-09,0.071009,0.039863,0.071821,0.069321
2020-04-13,-0.002860,-0.032633,-0.035795,-0.017435
...,...,...,...,...
2021-09-24,-0.017514,0.019896,0.008264,0.002024
2021-09-27,-0.001379,0.014597,0.009704,0.002585
2021-09-28,-0.009772,-0.006203,-0.017760,0.016994


In [10]:
pip install arch


In [11]:
from arch import arch_model
import pandas as pd
import numpy as np


In [12]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility


In [13]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols


Estimating GARCH volatility for IT sector...
Estimating GARCH volatility for Banking sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003928. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002982. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Pharma sector...
Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003609. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003777. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

In [14]:
print(garch_volatility_data['FMCG'].tail())


            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2021-09-24  0.012000       0.013703  0.017868
2021-09-27  0.011821       0.013844  0.017858
2021-09-28  0.013388       0.014028  0.017399
2021-09-29  0.013337       0.013514  0.016998
2021-09-30  0.013095       0.013834  0.016579


LSTM

In [15]:
pip install numpy pandas scikit-learn tensorflow


In [16]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler


In [17]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense



def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model





In [18]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility


In [19]:
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols


Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


In [20]:
print(gru_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2021-09-24    0.000902  0.001079  0.001613  0.001196
2021-09-27    0.000962  0.001312  0.001602  0.001191
2021-09-28    0.001000  0.001327  0.001591  0.001278
2021-09-29    0.001030  0.001327  0.001637  0.001523
2021-09-30    0.001194  0.001371  0.001723  0.001718


In [21]:
gru_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2020-05-07         NaN       NaN       NaN       NaN
2020-05-08         NaN       NaN       NaN       NaN
2020-05-11         NaN       NaN       NaN       NaN
2020-05-12         NaN       NaN       NaN       NaN
2020-05-13         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2021-09-24    0.000902  0.001079  0.001613  0.001196
2021-09-27    0.000962  0.001312  0.001602  0.001191
2021-09-28    0.001000  0.001327  0.001591  0.001278
2021-09-29    0.001030  0.001327  0.001637  0.001523
2021-09-30    0.001194  0.001371  0.001723  0.001718

[352 rows x 4 columns]>

In [22]:
gru_volatility_data['IT'].head()

,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2020-05-07,NaN,NaN,NaN,NaN
2020-05-08,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN
2020-05-12,NaN,NaN,NaN,NaN
2020-05-13,NaN,NaN,NaN,NaN


In [23]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']


/tmp/ipython-input-2331807048.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [24]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta



In [25]:
beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...


In [26]:
beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.774228  0.745206  0.695666  0.697242     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.731114     1.252223   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000     0.00000      0.000000   
Banking      1.746477      1.136998  0.000000     0.00000      0.000000   
Pharma       0.000000      0.000000  0.357097     0.36972      0.562616   
Auto         0.000000      0.000000  0.000000     0.00000      0.000000   
FMCG         0.000000      0.000000  0.000000     0.00000      0.000000   

           M&M.NS  MARUTI.NS  TATAMOTORS.NS  DABUR.NS  HINDUNILVR.NS    IT

Computing Sharpe Ratio

In [27]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes


In [28]:
features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]
            features.append([sector, stock, beta, garch_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe']
)


In [29]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'GRU_Vol', 'Sharpe'])


In [30]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [-0.03274432  5.34550759 -8.82973518]
Intercept: 0.04242978653241847


In [31]:
features_df['Risk_Score'] = X @ weights


In [32]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])



Top 3 safest stocks in IT:
        Stock  Risk_Score
2      TCS.NS    0.048394
1     INFY.NS    0.050339
0  HCLTECH.NS    0.067894

Top 3 safest stocks in Banking:
          Stock  Risk_Score
5   HDFCBANK.NS    0.030773
6  ICICIBANK.NS    0.051465
7  KOTAKBANK.NS    0.053892

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS    0.062001
9     DRREDDY.NS    0.064727
8       CIPLA.NS    0.077544

Top 3 safest stocks in Auto:
            Stock  Risk_Score
12      MARUTI.NS    0.056804
13  TATAMOTORS.NS    0.074002
11         M&M.NS    0.079329

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
16         ITC.NS    0.053207
15  HINDUNILVR.NS    0.054969
14       DABUR.NS    0.055307


GRU AND GARCH ONLY

In [33]:
from sklearn.linear_model import LinearRegression

X_new = features_df[['GARCH_Vol', 'GRU_Vol']]
y_new = features_df['Sharpe']

model_new = LinearRegression().fit(X_new, y_new)
weights_new = model_new.coef_
intercept_new = model_new.intercept_

print("New Weights (GARCH, GRU):", weights_new)


New Weights (GARCH, GRU): [  2.72281343 -10.56530611]


In [34]:
features_df['Risk_Score_GRU_GARCH'] = X_new @ weights_new


In [35]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + GRU only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_GARCH']])



Safest stocks in IT (GARCH + GRU only):
      Stock  Risk_Score_GRU_GARCH
2    TCS.NS              0.025080
1   INFY.NS              0.025539
3  WIPRO.NS              0.035896

Safest stocks in Banking (GARCH + GRU only):
          Stock  Risk_Score_GRU_GARCH
5   HDFCBANK.NS              0.016258
7  KOTAKBANK.NS              0.031105
6  ICICIBANK.NS              0.037694

Safest stocks in Pharma (GARCH + GRU only):
           Stock  Risk_Score_GRU_GARCH
9     DRREDDY.NS              0.022790
10  SUNPHARMA.NS              0.025251
8       CIPLA.NS              0.034212

Safest stocks in Auto (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
12      MARUTI.NS              0.032861
13  TATAMOTORS.NS              0.034973
11         M&M.NS              0.050708

Safest stocks in FMCG (GARCH + GRU only):
            Stock  Risk_Score_GRU_GARCH
16         ITC.NS              0.020252
15  HINDUNILVR.NS              0.028205
14       DABUR.NS              0.029144


GRU AND BETA ONLY

In [36]:
from sklearn.linear_model import LinearRegression

X_lb = features_df[['Beta', 'GRU_Vol']]
y_lb = features_df['Sharpe']

model_lb = LinearRegression().fit(X_lb, y_lb)
weights_lb = model_lb.coef_
intercept_lb = model_lb.intercept_

print("Weights: Beta =", weights_lb[0], ", GRU =", weights_lb[1])
print("Intercept:", intercept_lb)


Weights: Beta = -0.0014303852232574075 , GRU = -0.6297966986633239
Intercept: 0.10474039037599846


In [37]:
features_df['Risk_Score_GRU_Beta'] = X_lb @ weights_lb


In [38]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (BiLSTM + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Beta')
          .head(3)[['Stock', 'Risk_Score_GRU_Beta']])



Safest stocks in IT (BiLSTM + Beta only):
      Stock  Risk_Score_GRU_Beta
1   INFY.NS            -0.002367
3  WIPRO.NS            -0.002189
2    TCS.NS            -0.002155

Safest stocks in Banking (BiLSTM + Beta only):
          Stock  Risk_Score_GRU_Beta
4   AXISBANK.NS            -0.004530
6  ICICIBANK.NS            -0.004330
5   HDFCBANK.NS            -0.003898

Safest stocks in Pharma (BiLSTM + Beta only):
           Stock  Risk_Score_GRU_Beta
10  SUNPHARMA.NS            -0.002436
9     DRREDDY.NS            -0.002225
8       CIPLA.NS            -0.001678

Safest stocks in Auto (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
13  TATAMOTORS.NS            -0.005069
12      MARUTI.NS            -0.003112
11         M&M.NS            -0.002900

Safest stocks in FMCG (BiLSTM + Beta only):
            Stock  Risk_Score_GRU_Beta
16         ITC.NS            -0.002303
15  HINDUNILVR.NS            -0.001375
14       DABUR.NS            -0.001305


GARCH AND BETA ONLY

In [39]:
from sklearn.linear_model import LinearRegression

X_gb = features_df[['Beta', 'GARCH_Vol']]
y_gb = features_df['Sharpe']

model_gb = LinearRegression().fit(X_gb, y_gb)
weights_gb = model_gb.coef_
intercept_gb = model_gb.intercept_

print("Weights: Beta =", weights_gb[0], ", GARCH =", weights_gb[1])
print("Intercept:", intercept_gb)


Weights: Beta = -0.03537610945034612 , GARCH = 4.304080570414926
Intercept: 0.04553110712813725


In [40]:
features_df['Risk_Score_GARCH_Beta'] = X_gb @ weights_gb


In [41]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta only):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GARCH_Beta']])



Safest stocks in IT (GARCH + Beta only):
        Stock  Risk_Score_GARCH_Beta
2      TCS.NS               0.045789
1     INFY.NS               0.048500
0  HCLTECH.NS               0.056667

Safest stocks in Banking (GARCH + Beta only):
          Stock  Risk_Score_GARCH_Beta
5   HDFCBANK.NS               0.037281
6  ICICIBANK.NS               0.046380
7  KOTAKBANK.NS               0.051086

Safest stocks in Pharma (GARCH + Beta only):
           Stock  Risk_Score_GARCH_Beta
10  SUNPHARMA.NS               0.063263
9     DRREDDY.NS               0.067937
8       CIPLA.NS               0.072390

Safest stocks in Auto (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
12      MARUTI.NS               0.052860
11         M&M.NS               0.065025
13  TATAMOTORS.NS               0.078717

Safest stocks in FMCG (GARCH + Beta only):
            Stock  Risk_Score_GARCH_Beta
14       DABUR.NS               0.048074
15  HINDUNILVR.NS               0.048671
16         ITC.NS         

In [42]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']].head()


,Stock,Risk_Score,Risk_Score_GRU_GARCH,Risk_Score_GRU_Beta,Risk_Score_GARCH_Beta
0,HCLTECH.NS,0.067894,0.039834,-0.001903,0.056667
1,INFY.NS,0.050339,0.025539,-0.002367,0.048500
2,TCS.NS,0.048394,0.025080,-0.002155,0.045789
3,WIPRO.NS,0.070190,0.035896,-0.002189,0.063689
4,AXISBANK.NS,0.061454,0.040385,-0.004530,0.057071


Comparison per sector

In [43]:
def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [44]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
2,TCS.NS,0.048394
1,INFY.NS,0.050339
0,HCLTECH.NS,0.067894



🔹 Top 3 safest stocks using Risk_Score_GRU_GARCH:


,Stock,Risk_Score_GRU_GARCH
2,TCS.NS,0.025080
1,INFY.NS,0.025539
3,WIPRO.NS,0.035896



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


...

,Stock,Risk_Score_GRU_GARCH
9,DRREDDY.NS,0.022790
10,SUNPHARMA.NS,0.025251
8,CIPLA.NS,0.034212



🔹 Top 3 safest stocks using Risk_Score_GRU_Beta:


,Stock,Risk_Score_GRU_Beta
10,SUNPHARMA.NS,-0.002436
9,DRREDDY.NS,-0.002225
8,CIPLA.NS,-0.001678



🔹 Top 3 safest stocks using Risk_Score_GARCH_Beta:


,Stock,Risk_Score_GARCH_Beta
10,SUNPHARMA.NS,0.063263
9,DRREDDY.NS,0.067937
8,CIPLA.NS,0.072390


Comparison table

In [45]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_GRU_GARCH', 'Risk_Score_GRU_Beta', 'Risk_Score_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')


In [46]:
sector_ranking_table("FMCG")


,Stock,Risk_Score_Rank,Risk_Score_GRU_GARCH_Rank,Risk_Score_GRU_Beta_Rank,Risk_Score_GARCH_Beta_Rank
16,ITC.NS,1.0,1.0,1.0,3.0
15,HINDUNILVR.NS,2.0,2.0,2.0,2.0
14,DABUR.NS,3.0,3.0,3.0,1.0
